In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras import regularizers
import PIL.Image
import socket
from skimage import io
import face_alignment
import glob
import json

from tqdm import tqdm
import sys
#sys.path.insert(0, 'C:/Users/limju/libs')
from libs import umeyama
from libs import landmarkingAligning
from libs import landmarkMasking
import imutils
import math

Using TensorFlow backend.


In [2]:
#Input the directory of the folder containing the images that the User Wish to Predict on
TESTDIR = "frontal/"
cas_alt2 = cv2.CascadeClassifier("model/haarcascade_frontalface_alt2.xml")
prototxt_path = 'model/caffeFaceExtractionModel/deploy.prototxt.txt'
caffemodel_path = 'model/caffeFaceExtractionModel/weights.caffemodel'

# Read the Model
model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)
# Using this Caffe Model for Facial Detection
model

<dnn_Net 00000263AFF6A770>

In [3]:
def extract_frames(video, frame_rate):
    
    #Since extracting Frames > remove any images in all folders
    image_folder = glob.glob('images/*')
    for f in image_folder:
        os.remove(f)
        
    frontal_folder = glob.glob('frontal/*')
    for f in frontal_folder:
        os.remove(f)
    
    #folder path to videos
    vid = "videos/" + video
    
    selectedVid = cv2.VideoCapture(vid)
    success,image = selectedVid.read()
    video_name = str(video)
    count1 = 0
    count2 = 0

    fps = selectedVid.get(cv2.CAP_PROP_FPS)
    print("FPS : {0}".format(fps))
    frame_count = int(selectedVid.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps
    print("Duration : {0} seconds".format(duration))
    cNoFrames = int(frame_rate)

    while success:
        success,image = selectedVid.read()
        if not success:
            break

        if count1 % cNoFrames == 0:
            #folder path to store extract please make sure same path as extract_align_face 
            cv2.imwrite("images/%d.jpg" % count2, image) 
            count2 += 1

        count1 += 1
    print("A Total of %d frames were extracted!" % count2)

In [4]:
def extract_face():
    count = 0
    #replace directory with full body image folder
    for file in tqdm(os.listdir('images')):
        # Obtain both the file_name and its extension 
        file_name, file_extension = os.path.splitext(file)

        # Only perform face extraction if it is a proper picture file
        if (file_extension in ['.png','.jpg']):
            #Replace directory to path of aligned images
            image = cv2.imread('images/' + file)

            (h, w) = image.shape[:2]
            blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
            model.setInput(blob)
            detections = model.forward()

            # Identify each face
            for i in range(0, detections.shape[2]):
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                confidence = detections[0, 0, i, 2]
                # If confidence > 0.5, save face found as an actual face image
                if (confidence > 0.5):

                    frame = image[startY:endY, startX:endX]  
                    frame = imutils.resize(frame, height=256)
                    xBBox = endX - startX 

                    x_offsetScaled = float((256 - frame.shape[1])/2*xBBox/frame.shape[1])
                    frame = image[startY:endY, math.floor(startX-x_offsetScaled):math.ceil(endX+x_offsetScaled)]
                    frame = imutils.resize(frame, height=256)

                    if (frame.shape[1] != 256):
                        frame = frame[0:256,0:256]

                    count += 1 
                    #Replace directory of path to store aligned, (Path to aligned faces)
                    cv2.imwrite('frontal/'+ file, frame)

In [5]:
#Only use frontal face images as model was only trained against frontal faces
test_data = []
def frontal_face_check():
    IMG_SIZE = 256
    def create_testing_data():  
        #path = os.path.join(TESTDIR, "predict") # put path to aligned faces
        #for img in os.listdir(path):
        counter = 0
        for img in tqdm(os.listdir('frontal')):
            try:
                img_array = cv2.imread(os.path.join('frontal/',img))
                retval, faces_haar_alt2 = cv2.face.getFacesHAAR(img_array,"model/haarcascade_frontalface_alt2.xml")
                faces_haar_alt2 = np.squeeze(faces_haar_alt2)
                if((faces_haar_alt2.size <= 1) or (faces_haar_alt2.ndim is not 1)):
                    continue
                else:
                    test_data.append(img_array)
                    counter += 1
            except Exception as e:
                print(e)
                pass

    create_testing_data()

In [6]:
vid_image = input("Are you parsing image or video? [ image = 1, videos = 2 ]: ")
vid_image = int(vid_image)
if vid_image is 1:
    extract_face()
    frontal_face_check()
    if(test_data) is None:
        print("Image invalid")
elif vid_image is 2:
    try:
        video_input = input("Name of Video [Key with extension]: ")
        frame_rate = input("Extract One of out Nth Frames: ")
        extract_frames(video_input, frame_rate)
        extract_face()
        frontal_face_check()
    except:
        print("Input not valid please rerun the cell")
else:
    print("Option Given not valid please restart program.")

Are you parsing image or video? [ image = 1, videos = 2 ]: 1


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 22.65it/s]


In [7]:
IMG_SIZE = 256
val_x = np.array(test_data).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [8]:
#loading the model, Insert the model path into the line below
model_name = input("Name of model to use without extension: ")
loaded_model = tf.keras.models.load_model('model/' + model_name + '.h5')

#Displaying a summary of the model used
loaded_model.summary()

#splash_model_30epochs

Name of model to use without extension: splash_model_30epochs
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 250, 250, 32)      4736      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 125, 125, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 125, 125, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 119, 119, 64)      100416    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 59, 59, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 59, 59, 64)        0         
____________________________________________________________

In [9]:
#this line will help use the loaded model to predict against the entire image within the folder
predictions = loaded_model.predict(val_x, verbose = 1)


1/1 [==============================] - 0s 993us/step


In [10]:

if vid_image is 2:
    counter = 0
    total = 0
    for values in predictions:
        value = values.astype(int)
        total += 1
        if value == 1:
            counter += 1
    percent = (counter/total) * 100
    round(percent)
    print("This video chances of being a deepfake is : " + str(round(percent,2)) + "%")
else:
    #Predicted Images 1 is for DeepFake, Whereas 0 is for Real Images
    counter = 1
    for values in predictions:
        value = values.astype(int)
        print("Image", counter, " is: ", 'fake' if value is 1 else 'real')
    

Image 1  is:  real
